# 2.2.1 Data augmentation

In [1]:
import os
import re

import tensorflow as tf
from cifar10_input import read_cifar10

/home/saucecat/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## meta data

In [2]:
IMAGE_SIZE = 24
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

MOVING_AVERAGE_DECAY = 0.9999   
NUM_EPOCHS_PER_DECAY = 350.0   
LEARNING_RATE_DECAY_FACTOR = 0.1
INITIAL_LEARNING_RATE = 0.1
TOWER_NAME = 'tower'

# this is mostly for terminal run
FLAGS = tf.app.flags.FLAGS

# Basic model parameters.
tf.app.flags.DEFINE_integer('batch_size', 128,
                            """Number of images to process in a batch.""")
tf.app.flags.DEFINE_string('data_dir', 'cifar10_data',
                           """Path to the CIFAR-10 data directory.""")
tf.app.flags.DEFINE_boolean('use_fp16', False,
                            """Train the model using fp16.""")
tf.app.flags.DEFINE_string('train_dir', '/tmp/cifar10_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 10000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 10,
                            """How often to log results to the console.""")

## create data batch

In [4]:
def _generate_image_and_label_batch(image, label, min_queue_examples, batch_size, shuffle):
    # create a queue that shuffles the examples
    # and then read 'batch_size' images + labels from the example queue
    
    num_preprocess_threads = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
            [image, label], 
            batch_size=batch_size, 
            num_threads=num_preprocess_threads, 
            capacity=min_queue_examples + 3 * batch_size, 
            min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
            [image, label], 
            batch_size=batch_size, 
            num_threads=num_preprocess_threads, 
            capacity=min_queue_examples + 3 * batch_size)
        
    tf.summary.image('images', images)
    
    return images, tf.reshape(label_batch, [batch_size])

def distorted_inputs(data_dir, batch_size):
    # construct distorteed input for CIFAR training 
    filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in xrange(1, 6)]
    
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)
            
    # create a queue that produces the filenames to read
    filename_queue = tf.train.string_input_producer(filenames)
    
    # read examples from files in the filename queue
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)
    
    height = width = IMAGE_SIZE
    
    # image processing for training the network
    # many distortions applied to the image
    
    # randomly crop a [height, width] section of the image
    distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
    
    # randomly flip the image horizontally
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image, lower=0.2, upper=1.8)
    
    # substract off the mean and divide by the variance of the pixels
    float_image = tf.image.per_image_standardization(distorted_image)
    
    # set the shapes of tensors
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])
    
    # ensure that the random shuffling has good mixing properties
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN * min_fraction_of_examples_in_queue)
    print('Filling queue with %d CIFAR images before starting to train. '
          'This will take a few minutes. ' % min_queue_examples)
    
    # generate a batch of images and labels by building up a queue of examples
    return _generate_image_and_label_batch(float_image, read_input.label, 
                                           min_queue_examples, batch_size, shuffle=True)

# 2.2.2 Recognition model

In [5]:
def _variable_with_weight_decay(name, shape, stddev, wd):
    # create an initialized variable with weight decay
    initializer = tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32)
    var = tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)
    
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        # add l2 loss into global losses collection
        tf.add_to_collection('losses', weight_decay)
    
    return var

def _activation_summary(x):
    # tensorboard summary
    tensor_name = x.op.name
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity', tf.nn.zero_fraction(x))

In [6]:
def inference(images):
    # build the CIFAR-10 model
    # we instantiate all variables using tf.get_variable() instead of
    # tf.Variable() in order to share variables across multiple GPU training runs
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5, 3, 64], stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.get_variable('biases', [64], initializer=tf.constant_initializer(0.0), dtype=tf.float32)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)
        
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
    # local response normalization
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9., beta=0.75, name='norm1')
    
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5, 64, 64], stddev=5e-2, wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.get_variable('biases', [64], tf.float32, tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)
        
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        
    with tf.variable_scope('local3') as scope:
        reshape = tf.reshape(pool2, [FLAGS.batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384], stddev=0.04, wd=0.004)
        biases = tf.get_variable('biases', [384], tf.float32, tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        _activation_summary(local3)
        
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192], stddev=0.04, wd=0.004)
        biases = tf.get_variable('biases', [192], tf.float32, tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        _activation_summary(local4)
        
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES], stddev=1 / 192., wd=0.0)
        biases = tf.get_variable('biases', [NUM_CLASSES], tf.float32, tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        _activation_summary(softmax_linear)
        
    return softmax_linear

In [7]:
def calc_loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

def _add_loss_summaries(total_loss):
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])
    
    for l in losses + [total_loss]:
        tf.summary.scalar(l.op.name + '_raw', l)
        tf.summary.scalar(l.op.name, loss_averages.average(l))
    
    return loss_averages_op

In [8]:
def train(total_loss, global_step):
    num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / FLAGS.batch_size
    decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)
    
    # decay the learning rate exponentially based on the number of steps
    lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE, global_step, decay_steps, 
                                    LEARNING_RATE_DECAY_FACTOR, staircase=True)
    tf.summary.scalar('learning_rate', lr)
    
    loss_averages_op = _add_loss_summaries(total_loss)
    
    # compute gradients
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.GradientDescentOptimizer(lr)
        grads = opt.compute_gradients(total_loss)
    
    # apply gradients
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)
    
    # Add histograms for trainable variables.
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)

    # Add histograms for gradients.
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)
            
    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    return train_op 

## 2.2.3 Train model

In [9]:
import time
from datetime import datetime

def train_model():
    with tf.Graph().as_default():
        global_step = tf.contrib.framework.get_or_create_global_step()
        
        if not FLAGS.data_dir:
            raise ValueError('Please supply a data_dir')
            
        data_dir = os.path.join(FLAGS.data_dir, 'cifar-10-batches-bin')
        images, labels = distorted_inputs(data_dir=data_dir,batch_size=FLAGS.batch_size)
        
        logits = inference(images)
        loss = calc_loss(logits, labels)
        
        train_op = train(loss, global_step)
        
    
        class _LoggerHook(tf.train.SessionRunHook):
            """Logs loss and runtime."""

            def begin(self):
                self._step = -1
                self._start_time = time.time()

            def before_run(self, run_context):
                self._step += 1
                return tf.train.SessionRunArgs(loss)  # Asks for loss value.

            def after_run(self, run_context, run_values):
                if self._step % FLAGS.log_frequency == 0:
                    current_time = time.time()
                    duration = current_time - self._start_time
                    self._start_time = current_time

                    loss_value = run_values.results
                    examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
                    sec_per_batch = float(duration / FLAGS.log_frequency)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f sec/batch)')
                    print(format_str % (datetime.now(), self._step, loss_value, examples_per_sec, sec_per_batch))

        with tf.train.MonitoredTrainingSession(
            checkpoint_dir=FLAGS.train_dir,
            hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
                   tf.train.NanTensorHook(loss),
                   _LoggerHook()],
            config=tf.ConfigProto(
                log_device_placement=FLAGS.log_device_placement)) as mon_sess: 
            while not mon_sess.should_stop(): mon_sess.run(train_op)

In [10]:
train_model()

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes. 
INFO:tensorflow:Summary name conv1/weight_loss (raw) is illegal; using conv1/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv2/weight_loss (raw) is illegal; using conv2/weight_loss__raw_ instead.
INFO:tensorflow:Summary name local3/weight_loss (raw) is illegal; using local3/weight_loss__raw_ instead.
INFO:tensorflow:Summary name local4/weight_loss (raw) is illegal; using local4/weight_loss__raw_ instead.
INFO:tensorflow:Summary name softmax_linear/weight_loss (raw) is illegal; using softmax_linear/weight_loss__raw_ instead.
INFO:tensorflow:Summary name cross_entropy (raw) is illegal; using cross_entropy__raw_ instead.
INFO:tensorflow:Summary name total_loss (raw) is illegal; using total_loss__raw_ instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/cifar10

2018-04-28 18:25:36.540965: step 750, loss = 2.83 (2319.1 examples/sec; 0.055 sec/batch)
2018-04-28 18:25:37.098110: step 760, loss = 2.75 (2297.5 examples/sec; 0.056 sec/batch)
2018-04-28 18:25:37.672276: step 770, loss = 2.72 (2229.3 examples/sec; 0.057 sec/batch)
2018-04-28 18:25:38.228775: step 780, loss = 2.58 (2300.1 examples/sec; 0.056 sec/batch)
2018-04-28 18:25:38.778588: step 790, loss = 2.65 (2328.1 examples/sec; 0.055 sec/batch)
INFO:tensorflow:global_step/sec: 17.879
2018-04-28 18:25:39.504607: step 800, loss = 2.87 (1763.0 examples/sec; 0.073 sec/batch)
2018-04-28 18:25:39.907189: step 810, loss = 2.68 (3179.5 examples/sec; 0.040 sec/batch)
2018-04-28 18:25:40.427023: step 820, loss = 2.48 (2462.3 examples/sec; 0.052 sec/batch)
2018-04-28 18:25:40.959180: step 830, loss = 2.62 (2405.3 examples/sec; 0.053 sec/batch)
2018-04-28 18:25:41.487688: step 840, loss = 2.67 (2421.9 examples/sec; 0.053 sec/batch)
2018-04-28 18:25:42.024644: step 850, loss = 2.51 (2383.8 examples/sec

2018-04-28 18:26:25.439241: step 1630, loss = 2.06 (2411.2 examples/sec; 0.053 sec/batch)
2018-04-28 18:26:25.959212: step 1640, loss = 1.80 (2461.7 examples/sec; 0.052 sec/batch)
2018-04-28 18:26:26.512366: step 1650, loss = 1.55 (2314.0 examples/sec; 0.055 sec/batch)
2018-04-28 18:26:27.056459: step 1660, loss = 1.91 (2352.5 examples/sec; 0.054 sec/batch)
2018-04-28 18:26:27.577080: step 1670, loss = 1.80 (2458.6 examples/sec; 0.052 sec/batch)
2018-04-28 18:26:28.096862: step 1680, loss = 1.90 (2462.6 examples/sec; 0.052 sec/batch)
2018-04-28 18:26:28.622048: step 1690, loss = 1.85 (2437.2 examples/sec; 0.053 sec/batch)
INFO:tensorflow:global_step/sec: 18.7142
2018-04-28 18:26:29.307727: step 1700, loss = 1.78 (1866.8 examples/sec; 0.069 sec/batch)
2018-04-28 18:26:29.779748: step 1710, loss = 1.74 (2711.7 examples/sec; 0.047 sec/batch)
2018-04-28 18:26:30.317432: step 1720, loss = 1.73 (2380.6 examples/sec; 0.054 sec/batch)
2018-04-28 18:26:30.856277: step 1730, loss = 1.87 (2375.5 

2018-04-28 18:27:14.035515: step 2500, loss = 1.28 (1745.9 examples/sec; 0.073 sec/batch)
2018-04-28 18:27:14.490651: step 2510, loss = 1.42 (2812.4 examples/sec; 0.046 sec/batch)
2018-04-28 18:27:15.086365: step 2520, loss = 1.51 (2148.7 examples/sec; 0.060 sec/batch)
2018-04-28 18:27:15.651108: step 2530, loss = 1.49 (2266.5 examples/sec; 0.056 sec/batch)
2018-04-28 18:27:16.215847: step 2540, loss = 1.48 (2266.5 examples/sec; 0.056 sec/batch)
2018-04-28 18:27:16.773889: step 2550, loss = 1.36 (2293.8 examples/sec; 0.056 sec/batch)
2018-04-28 18:27:17.330995: step 2560, loss = 1.46 (2297.6 examples/sec; 0.056 sec/batch)
2018-04-28 18:27:17.886124: step 2570, loss = 1.50 (2305.7 examples/sec; 0.056 sec/batch)
2018-04-28 18:27:18.434566: step 2580, loss = 1.33 (2333.9 examples/sec; 0.055 sec/batch)
2018-04-28 18:27:18.995539: step 2590, loss = 1.35 (2281.8 examples/sec; 0.056 sec/batch)
INFO:tensorflow:global_step/sec: 17.5668
2018-04-28 18:27:19.728198: step 2600, loss = 1.48 (1747.0 

2018-04-28 18:28:03.881669: step 3380, loss = 1.10 (2311.4 examples/sec; 0.055 sec/batch)
2018-04-28 18:28:04.430994: step 3390, loss = 1.21 (2330.1 examples/sec; 0.055 sec/batch)
INFO:tensorflow:global_step/sec: 18.1564
2018-04-28 18:28:05.143587: step 3400, loss = 1.44 (1796.2 examples/sec; 0.071 sec/batch)
2018-04-28 18:28:05.574609: step 3410, loss = 1.08 (2969.7 examples/sec; 0.043 sec/batch)
2018-04-28 18:28:06.116900: step 3420, loss = 1.17 (2360.4 examples/sec; 0.054 sec/batch)
2018-04-28 18:28:06.686438: step 3430, loss = 1.15 (2247.4 examples/sec; 0.057 sec/batch)
2018-04-28 18:28:07.278142: step 3440, loss = 1.29 (2163.3 examples/sec; 0.059 sec/batch)
2018-04-28 18:28:07.863237: step 3450, loss = 1.27 (2187.7 examples/sec; 0.059 sec/batch)
2018-04-28 18:28:08.485201: step 3460, loss = 1.27 (2058.0 examples/sec; 0.062 sec/batch)
2018-04-28 18:28:09.050044: step 3470, loss = 1.32 (2266.1 examples/sec; 0.056 sec/batch)
2018-04-28 18:28:09.654630: step 3480, loss = 1.26 (2117.1 

...

INFO:tensorflow:global_step/sec: 16.886
2018-04-28 18:30:30.230984: step 6000, loss = 0.95 (1597.0 examples/sec; 0.080 sec/batch)
2018-04-28 18:30:30.679502: step 6010, loss = 0.89 (2853.9 examples/sec; 0.045 sec/batch)
2018-04-28 18:30:31.254502: step 6020, loss = 0.94 (2226.1 examples/sec; 0.058 sec/batch)
2018-04-28 18:30:31.819948: step 6030, loss = 0.99 (2263.7 examples/sec; 0.057 sec/batch)
2018-04-28 18:30:32.385834: step 6040, loss = 0.81 (2262.0 examples/sec; 0.057 sec/batch)
2018-04-28 18:30:32.924340: step 6050, loss = 1.08 (2376.9 examples/sec; 0.054 sec/batch)
2018-04-28 18:30:33.484735: step 6060, loss = 1.09 (2284.1 examples/sec; 0.056 sec/batch)
2018-04-28 18:30:34.063367: step 6070, loss = 0.94 (2212.1 examples/sec; 0.058 sec/batch)
2018-04-28 18:30:34.656172: step 6080, loss = 0.97 (2159.2 examples/sec; 0.059 sec/batch)
2018-04-28 18:30:35.216795: step 6090, loss = 1.01 (2283.2 examples/sec; 0.056 sec/batch)
INFO:tensorflow:global_step/sec: 17.5459
2018-04-28 18:30:35

2018-04-28 18:31:19.710439: step 6870, loss = 0.99 (2388.1 examples/sec; 0.054 sec/batch)
2018-04-28 18:31:20.280700: step 6880, loss = 0.87 (2244.6 examples/sec; 0.057 sec/batch)
2018-04-28 18:31:20.855191: step 6890, loss = 0.83 (2228.1 examples/sec; 0.057 sec/batch)
INFO:tensorflow:global_step/sec: 18.4182
2018-04-28 18:31:21.570604: step 6900, loss = 1.09 (1789.2 examples/sec; 0.072 sec/batch)
2018-04-28 18:31:21.992048: step 6910, loss = 0.81 (3037.2 examples/sec; 0.042 sec/batch)
2018-04-28 18:31:22.549011: step 6920, loss = 0.91 (2298.2 examples/sec; 0.056 sec/batch)
2018-04-28 18:31:23.107768: step 6930, loss = 0.91 (2290.8 examples/sec; 0.056 sec/batch)
2018-04-28 18:31:23.650452: step 6940, loss = 1.07 (2358.6 examples/sec; 0.054 sec/batch)
2018-04-28 18:31:24.186585: step 6950, loss = 0.94 (2387.5 examples/sec; 0.054 sec/batch)
2018-04-28 18:31:24.726134: step 6960, loss = 0.89 (2372.4 examples/sec; 0.054 sec/batch)
2018-04-28 18:31:25.279987: step 6970, loss = 0.83 (2311.1 

2018-04-28 18:32:08.492682: step 7740, loss = 0.90 (2244.6 examples/sec; 0.057 sec/batch)
2018-04-28 18:32:09.078174: step 7750, loss = 0.84 (2186.2 examples/sec; 0.059 sec/batch)
2018-04-28 18:32:09.650996: step 7760, loss = 1.06 (2234.5 examples/sec; 0.057 sec/batch)
2018-04-28 18:32:10.220777: step 7770, loss = 0.73 (2246.5 examples/sec; 0.057 sec/batch)
2018-04-28 18:32:10.827665: step 7780, loss = 0.97 (2109.1 examples/sec; 0.061 sec/batch)
2018-04-28 18:32:11.403470: step 7790, loss = 0.99 (2223.0 examples/sec; 0.058 sec/batch)
INFO:tensorflow:global_step/sec: 17.2359
2018-04-28 18:32:12.150464: step 7800, loss = 0.85 (1713.5 examples/sec; 0.075 sec/batch)
2018-04-28 18:32:12.653345: step 7810, loss = 0.90 (2545.4 examples/sec; 0.050 sec/batch)
2018-04-28 18:32:13.224127: step 7820, loss = 0.87 (2242.5 examples/sec; 0.057 sec/batch)
2018-04-28 18:32:13.837416: step 7830, loss = 0.99 (2087.1 examples/sec; 0.061 sec/batch)
2018-04-28 18:32:14.431870: step 7840, loss = 1.01 (2153.2 

2018-04-28 18:32:59.887953: step 8610, loss = 0.90 (3090.0 examples/sec; 0.041 sec/batch)
2018-04-28 18:33:00.425808: step 8620, loss = 0.98 (2379.8 examples/sec; 0.054 sec/batch)
2018-04-28 18:33:00.955969: step 8630, loss = 0.91 (2414.3 examples/sec; 0.053 sec/batch)
2018-04-28 18:33:01.487693: step 8640, loss = 0.98 (2407.3 examples/sec; 0.053 sec/batch)
2018-04-28 18:33:02.021098: step 8650, loss = 1.03 (2399.7 examples/sec; 0.053 sec/batch)
2018-04-28 18:33:02.544187: step 8660, loss = 0.86 (2447.0 examples/sec; 0.052 sec/batch)
2018-04-28 18:33:03.091464: step 8670, loss = 0.82 (2338.9 examples/sec; 0.055 sec/batch)
2018-04-28 18:33:03.664787: step 8680, loss = 0.87 (2232.6 examples/sec; 0.057 sec/batch)
2018-04-28 18:33:04.194196: step 8690, loss = 1.07 (2417.8 examples/sec; 0.053 sec/batch)
INFO:tensorflow:global_step/sec: 18.5428
2018-04-28 18:33:04.866682: step 8700, loss = 1.07 (1903.4 examples/sec; 0.067 sec/batch)
2018-04-28 18:33:05.286266: step 8710, loss = 1.11 (3050.7 

2018-04-28 18:33:50.134198: step 9490, loss = 0.81 (2087.3 examples/sec; 0.061 sec/batch)
INFO:tensorflow:global_step/sec: 16.9881
2018-04-28 18:33:50.859934: step 9500, loss = 0.97 (1763.7 examples/sec; 0.073 sec/batch)
2018-04-28 18:33:51.276703: step 9510, loss = 0.73 (3071.3 examples/sec; 0.042 sec/batch)
2018-04-28 18:33:51.828483: step 9520, loss = 0.87 (2319.8 examples/sec; 0.055 sec/batch)
2018-04-28 18:33:52.391381: step 9530, loss = 0.84 (2274.0 examples/sec; 0.056 sec/batch)
2018-04-28 18:33:52.967313: step 9540, loss = 0.92 (2222.5 examples/sec; 0.058 sec/batch)
2018-04-28 18:33:53.528599: step 9550, loss = 1.09 (2280.5 examples/sec; 0.056 sec/batch)
2018-04-28 18:33:54.059802: step 9560, loss = 0.99 (2409.6 examples/sec; 0.053 sec/batch)
2018-04-28 18:33:54.638070: step 9570, loss = 0.81 (2213.5 examples/sec; 0.058 sec/batch)
2018-04-28 18:33:55.172598: step 9580, loss = 0.75 (2394.6 examples/sec; 0.053 sec/batch)
2018-04-28 18:33:55.695503: step 9590, loss = 1.01 (2447.9 